# ETL para construcción de funciones de la APP

In [1]:
# Importando librerías
import pandas as pd 
from openpyxl import load_workbook

In [ ]:
df_iva = pd.read_excel("IVA.xlsx", names= ['Numero RUC', 'Nombre y Apellido o Razon Social', 'Numero Documento',
       'Descripcion del Pago', 'Fecha de Emision de Documento',
       'Ingreso sin IVA', 'Monto IVA Trasladado', 'Codigo Renglon'], engine="openpyxl")
df_iva.head()


### Verificación de RUC

In [5]:
def RucVerification (df):
    df_RUC = df.copy()
    df_RUC["Has14"] = df_RUC["Numero RUC"].map(len)!=14.00 # Cada RUC debe tener 14 carácteres
    df_result = df_RUC[df_RUC["Has14"] == True].drop(["Has14"],axis=1) # Filtra el dataframe para mostrar los registros que no tengan 14 caracteres
    result = None
    if df_result.empty:
        result = "Todos los campos del RUC tienen los 14 caracteres" # Resultado si las filas están correctas
    else:
        result = df_result # Resultado si las filas no están correctas
    
    return result


### Verificación de Número de Documento

In [8]:
df_docnumber = df_iva.copy()

df_docnumber["DocEsNo"] = pd.to_numeric(df_iva["Numero Documento"], errors='coerce').isnull()


In [ ]:
df_docnumber[df_docnumber["DocEsNo"] == True].drop("DocEsNo", axis=1)

In [13]:
def DocumentVerification (df):
    
    
    df_docnumber  = df.copy()
    df_docnumber["DocEsNo"] = pd.to_numeric(df["Numero Documento"], errors='coerce').isnull() 
    df_result = df_docnumber[df_docnumber["DocEsNo"] == True].drop("DocEsNo", axis=1)
    

    if df_result.empty:
        result = "Los documentos no contienen letras" # Resultado si las filas están correctas
    else:
        result = df_result # Resultado si las filas no están correctas
    
    return result

### Verificacion de Fecha

In [1]:
def DateVerification (df):
    df_date = df.copy()
    
    def is_date(value):
        try:
            pd.to_datetime(value,errors='raise')
            return True
        except ValueError:
            return False

    df_date["is_date"] = df_date["Fecha de Emision de Documento"].apply(is_date)
    df_date["Fecha de Emision de Documento"] = pd.to_datetime(df_date["Fecha de Emision de Documento"], errors='coerce')
    df_date["Año"] = df_date["Fecha de Emision de Documento"].dt.year
    df_result = df_date[(df_date["Año"] != pd.Timestamp.today().year) | 
                        (df_date["is_date"] == False)].drop("Año", axis = 1)
    result = df_result

    return result

In [ ]:
DateVerification(df_iva)

### Verificación de Monto e IVA

#### Monto Verificación

In [53]:
ws = load_workbook("IVA.xlsx", data_only=False).active
data = list(ws.values)
df_monto = pd.DataFrame(data[1:], columns=data[0])

In [12]:
def VerificacionFormula (file):
    
    #Sub-función para rastrear formulas 
    def contains_equal (celda):
        return '=' in str(celda)
    
    #Leyendo el archivo con el motor Openpyxl para captar las celdas con formulas
    ws = load_workbook(file, data_only=False).active
    data = list(ws.values)
    df_monto = pd.DataFrame(data[1:], columns=data[0])

    df_bool = df_monto.applymap(contains_equal)
    df_result = df_monto[df_bool.any(axis=1)]

    result = None

    if df_result.empty:
        result = "Ninguna celda en el archivo tiene fórmulas ✅"
    else:
        result = df_result[df_result.columns[:8]]

    return result

    

In [ ]:
VerificacionFormula("IVA.xlsx")

#### IVA Verificación

In [24]:
def VerificacionIVA (df):
    df_calculation = df
    df_calculation["Calculo IVA"] = round(df_calculation["Monto IVA Trasladado"]/df_calculation["Ingreso sin IVA"],2)
    df_calculation["% Calculo IVA"] = (df_calculation["Calculo IVA"] * 100).astype(str) + '%'
    df_result = df_calculation[df_calculation["Calculo IVA"] != 0.15].drop("Calculo IVA",axis=1)

    if df_result.empty:
        result = "El IVA está calculado correctamente (15% del monto) ✅"
    else:
        result = df_result

    return result

In [ ]:
VerificacionIVA(df_iva)

### Codigo Renglon Verificación

In [23]:
def VerificacionCodigo (df):
    df_codigo = df.copy()
    df_codigo["Verificacion"] = df_codigo["Descripcion del Pago"] + df_codigo["Codigo Renglon"].astype(str)
    df_result = df_codigo[~df_codigo["Verificacion"].isin(["Compra11","Servicios13"])]


    result = None

    if df_result.empty:
        result = "Los codigos y descripciones coinciden ✅"
    else:
        result = df_result

    return result

    



In [ ]:
VerificacionCodigo(df_iva)